In [ ]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import photogrammetry as pg

# plt.style.use('seaborn-poster')
plt.style.use('default')

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
# in case a new board type is needed
# pg.generate_charuco_board(pg.BOARD_SQUARE_SIZE, pg.BOARD_ARUCO_SIZE, aruco_dict=cv2.aruco.DICT_4X4_1000, gen_png=True)

In [ ]:
# check for pickled camera matrices to avoid expensive recalibration
cal_path = os.path.join(os.path.expanduser('~'), 'tmp', 'calib')

extensions = ['.jpeg', '.jpg', '.JPG', '.tiff', '.TIFF']
files = []
for ext in extensions:
    files += sorted(glob.glob(os.path.join(cal_path, '**' + ext)))
assert files, 'No image files found when searching for images for camera calibration.'

if not (
    os.path.exists(os.path.join(cal_path, 'camera_cal_mtx.pickle')) and
    os.path.exists(os.path.join(cal_path, 'camera_cal_dist.pickle')) and
    os.path.exists(os.path.join(cal_path, 'camera_cal_optimal_camera_matrix.pickle'))
):
    # calibrate the camera for distortion
    mtx, dist, optimal_camera_matrix, roi = pg.calibrate_camera(
        files,
        square_size=pg.BOARD_SQUARE_SIZE,
        aruco_size=pg.BOARD_ARUCO_SIZE,
        plot=False
    )

    with open(os.path.join(cal_path, 'camera_cal_mtx.pickle'), 'wb') as f:
        pickle.dump(mtx, f, protocol=pickle.HIGHEST_PROTOCOL)
    with open(os.path.join(cal_path, 'camera_cal_dist.pickle'), 'wb') as f:
        pickle.dump(dist, f, protocol=pickle.HIGHEST_PROTOCOL)
    with open(os.path.join(cal_path, 'camera_cal_optimal_camera_matrix.pickle'), 'wb') as f:
        pickle.dump(optimal_camera_matrix, f, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open(os.path.join(cal_path, 'camera_cal_mtx.pickle'), 'rb') as f:
        mtx = pickle.load(f)
    with open(os.path.join(cal_path, 'camera_cal_dist.pickle'), 'rb') as f:
        dist = pickle.load(f)
    with open(os.path.join(cal_path, 'camera_cal_optimal_camera_matrix.pickle'), 'rb') as f:
        optimal_camera_matrix = pickle.load(f)
print(mtx)
print(dist)
print(optimal_camera_matrix)

In [ ]:
meas_path = os.path.join(os.path.expanduser('~'), 'tmp', 'meas')
files = []
for ext in extensions:
    files += sorted(glob.glob(os.path.join(meas_path, '**' + ext)))
assert files, 'No image files found when searching for images for camera calibration.'

In [ ]:
# use camera cal matrix to de-distort a few images to check
for file in files[:3]:
    fig, ax = plt.subplots()
    ax.imshow(pg.load_to_gray(file, optimal_camera_matrix, dist), cmap='bone')

In [ ]:
%autoreload

img_data = pg.measure_images(files, optimal_camera_matrix, dist, aruco_ids=[46], plot=False)

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection':'3d'})

for file, entities in img_data.items():
    for id, pose in entities.items():
        for key, val in pose.items():
            print(file, id, key, val)
            if key == 'tvec':
                ax.scatter(val[0], val[1], val[2], label=id)
ax.legend()
ax.set_box_aspect([1,1,1])
ax.set_xlim(-.2,.2)
ax.set_ylim(-.2,.2)
ax.set_zlim(-.2,.2)

In [ ]:
for file in files:
    if file in img_data.keys():
        theta = pg.estimate_angular_offset(img_data, file, 'board', 46)
        print(np.rad2deg(theta))

In [ ]:
%autoreload

commanded_pts, actual_points, residuals = pg.post_process_scan(
    img_data,
    '/home/evanmayer/tmp/calib/',
    os.path.join('..', 'data', 'input', 'profiles', '24in_breadboard_raster_skipline_10x10_0.30mx0.30m.csv'),
    raft_id=46
)